In [1]:
import pandas as pd
import random
import inflect
from IPython.display import display

## Pre-Draft Proceedings

Set parameters

In [2]:
last_yr = 'Draft Results 2017.xlsx'
last_yr_indv = 'Individual Draft Results 2017.xlsx'

Read last year's Draft Results into Pandas DataFrame

In [3]:
last_yr_df = pd.read_excel(last_yr, index_col=3)

Determine keepers

In [4]:
owners = pd.ExcelFile(last_yr_indv).sheet_names
keepers = {}
for owner in owners:
    input_str = '\n\n{}, who would you like to keep? '.format(owner)
    player = input(input_str)
    while True:
        if player in last_yr_df.index:
            if last_yr_df.Round[player] > 1:
                round_lost = last_yr_df.Round[player] - 1
                break
            else:
                input_str = '\nYou drafted that player in the 1st Round and ' \
                'cannot keep them. Who else would you like to keep? '
                player = input(input_str)
        else:
            round_lost = 16
            break
    print('\n{} will count as your {} Round pick.'.format(
        player, inflect.engine().ordinal(round_lost)))
    keepers[owner] = {'player': player, 'round': round_lost}



Garrett Jung, who would you like to keep? Travis Kelce

Travis Kelce will count as your 8th Round pick.


Alex Abdella, who would you like to keep? Kareem Hunt

Kareem Hunt will count as your 10th Round pick.


Max Abdella, who would you like to keep? Derrick Henry

Derrick Henry will count as your 9th Round pick.


Andrew McGee, who would you like to keep? Melvin Gordon

Melvin Gordon will count as your 6th Round pick.


Grant Franke, who would you like to keep? Alvin Kamara

Alvin Kamara will count as your 16th Round pick.


Andrew Bender, who would you like to keep? Ezekiel Elliott

Ezekiel Elliott will count as your 3rd Round pick.


Owen Johnson, who would you like to keep? Adam Thielen

Adam Thielen will count as your 12th Round pick.


Joe Schwieters, who would you like to keep? Harrison Butker

Harrison Butker will count as your 16th Round pick.


Determine draft order

In [5]:
random.shuffle(owners)
draft_order = [None] * len(owners)
for owner in owners:
    input_str = "\n{}, you're up!\nWhich draft slot would you like? ".format(
        owner)
    slot = int(input(input_str))
    while True:
        if draft_order[slot - 1]:
            input_str = '\nThat draft slot is already taken. Pick a ' \
            'different one: '
            slot = int(input(input_str))
        else:
            draft_order[slot - 1] = owner
            break


Alex Abdella, you're up!
Which draft slot would you like? 8

Max Abdella, you're up!
Which draft slot would you like? 7

Andrew McGee, you're up!
Which draft slot would you like? 2

Grant Franke, you're up!
Which draft slot would you like? 4

Owen Johnson, you're up!
Which draft slot would you like? 1

Joe Schwieters, you're up!
Which draft slot would you like? 5

Andrew Bender, you're up!
Which draft slot would you like? 3

Garrett Jung, you're up!
Which draft slot would you like? 6


## Draft

Load player pool

In [6]:
player_pool = pd.read_excel('Raw Data 2018.xlsx', index_col=[0], usecols=[0, 1, 2])
player_pool['Position'] = player_pool['Position'].str.strip()

Set draft parameters

In [7]:
num_rounds = 16

Initialize draft history

In [8]:
num_picks = len(draft_order) * num_rounds
column_names = ['Round', 'Player', 'Position', 'Owner']
draft_history = pd.DataFrame(index=[], columns=column_names)
draft_history.index.name = 'Pick Overall'

Initialize individual draft histories and depth charts

In [9]:
draft_history_indv = {}
depth_charts ={}
for owner in draft_order:
    column_names = ['Round', 'Player', 'Position']
    draft_history_indv[owner] = pd.DataFrame(index=[], columns=column_names)
    draft_history_indv[owner].index.name = 'Pick Overall'
    depth_charts[owner] = pd.read_excel('Blank Depth Chart.xlsx', index_col=[0])

Draft helper function - fill depth chart

In [10]:
def fill_depth_chart(owner, position):
    spots = depth_charts[owner].index.tolist()
    for spot in spots:
        if (position or 'Bench') in spot:
            if pd.isnull(depth_charts[owner].at[spot, 'Player']):
                index = spot
                break
    return index

Perform draft

In [11]:
pick = 1
for round_num in range(1, num_rounds + 1):
    print('\n\n\n\n\nROUND {}'.format(round_num))
    for owner in draft_order:
        print("\n\n\n\n\n{}, you're up!".format(owner))
        while True:
            input_str = """
You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

"""
            # Check if keeper should be taken this round
            if keepers[owner]['round'] == round_num:
                option = keepers[owner]['player']
            else:
                option = input(input_str)
            
            if option == '1':
                display(draft_history_indv[owner])
            #elif option == '2':
                #display(depth_charts[owner])
            elif option == '2':
                display(draft_history)
            else:
                player = option
                while True:
                    if player in player_pool.index.tolist():
                        the_pick = player_pool.loc[player]
                        player_pool.drop(player)
                        break
                    player = input('\nThat player is not in the player pool. Please re-enter ' \
                                   'the player, making sure you spelled his name correctly: ')

                # Update depth chart / draft histories
                draft_history.loc[pick] = [str(round_num), player, the_pick['Position'], owner]
                draft_history_indv[owner].loc[pick] = [str(round_num), player, the_pick['Position']]
                #index = fill_depth_chart(owner, the_pick['Position'])
                #depth_charts[owner].loc[index] = player
                
                # Display pick
                print('\n\n\n\n\n{} Took {} with the {} Overall Pick'.format(
                    owner, player,  inflect.engine().ordinal(pick)))
                pick += 1
                break
    
    # Save excel spreedsheets
    writer = pd.ExcelWriter('Draft Results 2018.xlsx')
    draft_history.to_excel(writer, 'Draft Results')
    writer.save()
    
    writer2 = pd.ExcelWriter('Individual Draft Results 2018.xlsx')
    for owner, df in draft_history_indv.items():
        df.to_excel(writer2, owner)
    writer2.save()
    
    #writer3 = pd.ExcelWriter('Individual Depth Charts 2018.xlsx')
    #for owner, df in depth_charts.items():
        #df.to_excel(writer3, owner)
    #writer3.save()
    
    draft_order.reverse()






ROUND 1





Owen Johnson, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Todd Gurley II





Owen Johnson Took Todd Gurley II with the 1st Overall Pick





Andrew McGee, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Le'Veon Bell





Andrew McGee Took Le'Veon Bell with the 2nd Overall Pick





Andrew Bender, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

David Johnson





Andrew Bender Took David Johnson with the 3rd Overall Pick





Grant Franke, you're up!

You can either ent

,Round,Player,Position,Owner
Pick Overall,,,,
1,1,Todd Gurley II,RB,Owen Johnson
2,1,Le'Veon Bell,RB,Andrew McGee
3,1,David Johnson,RB,Andrew Bender
4,1,Antonio Brown,WR,Grant Franke
5,1,Saquon Barkley,RB,Joe Schwieters
6,1,Leonard Fournette,RB,Garrett Jung
7,1,Dalvin Cook,RB,Max Abdella
8,1,Julio Jones,WR,Alex Abdella
9,2,Odell Beckham Jr.,WR,Alex Abdella



You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Joe Mixon





Alex Abdella Took Joe Mixon with the 24th Overall Pick





ROUND 4





Alex Abdella, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Jerrick McKinnon

That player is not in the player pool. Please re-enter the player, making sure you spelled his name correctly: Jerick McKinnon





Alex Abdella Took Jerick McKinnon with the 25th Overall Pick





Max Abdella, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Josh Gordon





Max Abdella Too

,Round,Player,Position
Pick Overall,,,
7,1,Dalvin Cook,RB
10,2,DeAndre Hopkins,WR
23,3,Tyreek Hill,WR
26,4,Josh Gordon,WR
39,5,Royce Freeman,RB



You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Golden Tate





Max Abdella Took Golden Tate with the 42nd Overall Pick





Garrett Jung, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Lamar Miller





Garrett Jung Took Lamar Miller with the 43rd Overall Pick





Joe Schwieters, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Alshon Jeffrey

That player is not in the player pool. Please re-enter the player, making sure you spelled his name correctly: Alshon Jeffery





Joe Schwieters Took Alshon 

...


You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Kyle Rudolph





Andrew McGee Took Kyle Rudolph with the 111th Overall Pick





Owen Johnson, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

DaVante Parker

That player is not in the player pool. Please re-enter the player, making sure you spelled his name correctly: Davante Parker

That player is not in the player pool. Please re-enter the player, making sure you spelled his name correctly: DeVante Parker





Owen Johnson Took DeVante Parker with the 112th Overall Pick





ROUND 15





Owen Johnson, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it

,Round,Player,Position
Pick Overall,,,
7,1,Dalvin Cook,RB
10,2,DeAndre Hopkins,WR
23,3,Tyreek Hill,WR
26,4,Josh Gordon,WR
39,5,Royce Freeman,RB
42,6,Golden Tate,WR
55,7,Chris Carson,RB
58,8,Emmanuel Sanders,WR
71,9,Derrick Henry,RB



You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Jack Doyle





Max Abdella Took Jack Doyle with the 119th Overall Pick





Alex Abdella, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Matt Ryan





Alex Abdella Took Matt Ryan with the 120th Overall Pick





ROUND 16





Alex Abdella, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Justin Tucker





Alex Abdella Took Justin Tucker with the 121st Overall Pick





Max Abdella, you're up!

You can either enter who you would like to draft or perform

,Round,Player,Position
Pick Overall,,,
2,1,Le'Veon Bell,RB
15,2,Stefon Diggs,WR
18,3,A.J. Green,WR
31,4,Kenyan Drake,RB
34,5,Larry Fitzgerald,WR
47,6,Melvin Gordon,RB
50,7,Dion Lewis,RB
63,8,Russell Wilson,QB
66,9,Greg Olsen,TE



You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

David Njoku





Andrew McGee Took David Njoku with the 127th Overall Pick





Owen Johnson, you're up!

You can either enter who you would like to draft or perform any of the following
options by entering it's corresponding number:

1) Look at who you already have drafted
2) Look at the full draft history

Stephen Gostkowski





Owen Johnson Took Stephen Gostkowski with the 128th Overall Pick
